## Introduction ##

In this notebook, I will implement a simple Neural Network with Theano to predict weather patterns based on the information contained on the Ocean Ships Logbooks. Obviously, real weather prediction requires many complex algorithms and the information found in the logbooks is very limited, but it can still be interesting to see what kind of results we can get out of it.

Before starting with the neural network, however, let's start having a look and see what kind of data we are working with.

Ship's Courses
--------------

The simplest analysis we can apply to this interesting dataset is trying to plot the courses followed by the different ships. Mapping it as a function of different features, such as the nationality of the ship, its origin port and its destination, allows us to infer the story of these vessels' voyages. 

In [ ]:
#Let's start by importing a few libraries we will need for the analysis

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cartopy.crs as ccrs 
from geopy.geocoders import Nominatim 

#And then import the database
df = pd.read_csv('../input/CLIWOC15.csv', low_memory=False)
#we remove Nan entries for the columns we are interested in studying
df1=df.dropna(subset = ['Lon3', 'Lat3', 'UTC', 'VoyageIni'])
df2=df.dropna(subset = ['VoyageFrom', 'VoyageTo', 'VoyageIni'])

Now we define a function that will plot all the courses for a given ship on a map built with Cartopy, after grouping them according to the starting date of the trip. 
We also define a function to plot the ports of origin and destination as spheres, with their size proportional to the number of visits, unfortunately, the geolocator is not working at the moment, so we'll skip this part of the analysis.

In [ ]:
def plotPath(ship,df,ax,col='#444444',alp=0.5):
    path=df[df['ShipName']==ship][['Lon3', 'Lat3', 'UTC', 'VoyageIni']]
    #Grouping the paths according to the start date of the Voyage helps splitting them
    #and plotting them properly, to be even more accurate one should introduce a function 
    #to split the voyages that present "jumps" from one part to another of the globe
    #which give rise to unpleasant lines crossing the land portions of the map,
    #but since this is just an exploratory plot we will ignore this for now
    groupedPath=path.groupby('VoyageIni')
    for name, group in groupedPath:
        if group.size<2:
            continue
        group.sort_values(by='UTC', inplace=True)
        #draw path on the background
        x,y=group['Lon3'].tolist(),group['Lat3'].tolist()
        ax.plot(x,y,color=col,alpha=alp,transform=ccrs.Geodetic(),linewidth=0.5)
        
#we define a function that finds all origin and destination ports and put them in a dictionary with 
#a count of how many ships departed from/arrived there

def plotPorts(ship,df,ax,col1='#444444',col2='#444444'):
    dictFrom, dictTo={},{}
    path=df[df['ShipName']==ship][['VoyageFrom', 'VoyageTo', 'VoyageIni']]
    groupedFrom=path.groupby('VoyageFrom')
    groupedTo=path.groupby('VoyageTo')
    for name, group in groupedFrom:
        place=group['VoyageFrom'].iloc[0]
        if place not in dictFrom:
            #here we locate the origin port from its name where possible
            location = geolocator.geocode(place)
            if location is not None:
                dictFrom[place]= [group.VoyageIni.nunique(), location.longitude, location.latitude]
            else:
                dictFrom[place][0]=dictFrom[place][0]+group.VoyageIni.nunique()
        for name, group in groupedTo:
            place=group['VoyageTo'].iloc[0]
            if place not in dictTo:
                #here we locate the arrival port from its name where possible
                location = geolocator.geocode(place)
                if location is not None:	
                    dictTo[place]= [group.VoyageIni.nunique(), location.longitude, location.latitude]
            else:
                dictTo[place][0]=dictTo[place][0]+group.VoyageIni.nunique()

    #now we generate the plot, with the ports as circles whose radius is proportional to their 
    #relative number of visits
    sumF= sum([v[0] for k,v in dictFrom.items()])
    sumT= sum([v[0] for k,v in dictTo.items()])
    maxA= max([v[0] for k,v in dictFrom.items()]+[v[0] for k,v in dictTo.items()])
        
    for el in dictFrom[:10]:
	    ax.plot(dictFrom[el][1],dictFrom[el][2],color=col1,transform=ccrs.Geodetic(),marker='o', ms=dictFrom[el][0]*20/maxA , mec='#222222', mew=0.5)
    for el in dictTo[:10]:
        ax.plot(dictTo[el][1],dictTo[el][2],color=col2,transform=ccrs.Geodetic(),marker='o', ms=dictTo[el][0]*20/maxA , mec='#222222', mew=0.5)

In [ ]:
#################WARNING##############################################################
#The following is a very basic workaround to the problem of not being able to download 
#the coastlines information with Cartopy on Kaggle's Notebooks
#I build a very low resolution version of the 1:110m map downloaded from
#http://www.naturalearthdata.com/ and transformed it with pyshp into a list
#This is a very dumb way to do this, the map is broken and
#very low resolution, but it's a quick temporary
#solution in case you are using Kaggle Notebooks
shapes=[[[141.0, -2.6], [146.97, -6.72], [150.8, -10.29], [146.05, -8.07], [140.14, -8.3], [135.99, -4.55], [133.07, -2.46], [132.38, -0.37], [139.18, -2.05]], [[114.2, 4.53], [118.35, 5.71], [117.88, 1.83], [116.0, -3.66], [110.22, -2.93], [111.17, 1.85]], [[-88.15, 74.39], [-96.75, 77.16], [-87.84, 75.57], [-81.95, 74.44]], [[-82.27, 23.19], [-76.52, 21.21], [-75.63, 19.87], [-79.28, 21.56], [-83.91, 22.15], [-82.51, 23.08]], [[-55.6, 51.32], [-53.48, 49.25], [-53.96, 47.63], [-59.42, 47.9], [-55.6, 51.32]], [[-83.88, 65.11], [-83.11, 64.1], [-85.16, 65.66]], [[-78.77, 72.35], [-67.91, 70.12], [-63.92, 65.0], [-64.67, 63.39], [-71.02, 62.91], [-77.9, 65.31], [-74.84, 68.55], [-84.94, 69.97], [-88.41, 73.54], [-78.77, 72.35]], [[-107.82, 75.85], [-113.87, 74.72], [-109.07, 75.47]], [[-121.54, 74.45], [-120.46, 71.38], [-121.54, 74.45]], [[49.54, -12.47], [49.67, -15.71], [47.55, -23.78], [43.35, -22.78], [44.04, -18.33], [46.88, -15.21], [49.54, -12.47]], [[-48.66, -78.05], [-44.88, -80.34], [-50.99, -79.61]], [[-73.92, -71.27], [-69.72, -69.25], [-69.96, -72.31], [-73.92, -71.27]], [[151.3, -5.84], [149.85, -5.51], [152.14, -4.15]], [[176.89, -40.07], [173.82, -39.51], [173.05, -35.24], [175.81, -36.8], [177.21, -39.15]], [[169.67, -43.56], [173.25, -41.33], [172.31, -43.87], [166.68, -46.22]], [[147.69, -40.81], [146.05, -43.55], [147.69, -40.81]], [[126.15, -32.22], [120.58, -33.93], [115.56, -34.39], [115.16, -30.6], [113.34, -26.12], [113.39, -24.38], [115.46, -21.5], [119.25, -19.95], [123.01, -16.41], [125.17, -14.68], [128.36, -14.87], [131.22, -12.18], [134.39, -12.04], [136.31, -13.29], [137.58, -16.22], [141.27, -16.39], [141.69, -12.41], [143.16, -12.33], [145.27, -15.43], [148.18, -19.96], [150.9, -23.46], [153.51, -29.0], [151.01, -34.31], [147.38, -38.22], [142.75, -38.54], [138.12, -35.61], [137.81, -32.9], [134.27, -32.62]], [[122.93, 0.88], [120.18, 0.24], [121.51, -1.9], [121.74, -4.85], [119.8, -5.67], [119.83, 0.15]], [[108.49, -6.42], [114.56, -8.75], [106.45, -7.35]], [[104.37, -1.08], [103.87, -5.04], [98.6, 1.82], [98.37, 4.27], [103.44, -0.71]], [[126.38, 8.41], [124.22, 6.16], [122.31, 8.03], [126.22, 9.29]], [[141.88, 39.18], [135.79, 33.46], [130.69, 31.03], [132.62, 35.43], [139.88, 40.56]], [[-4.21, 58.55], [-1.11, 54.62], [0.55, 50.77], [-4.31, 51.21], [-2.95, 53.98], [-6.15, 56.79]], [[-14.51, 66.46], [-21.78, 64.4], [-19.06, 66.28]], [[142.91, 53.7], [143.51, 46.14], [141.59, 51.94]], [[122.34, 18.22], [122.7, 14.34], [122.03, 13.78], [119.92, 15.41], [122.34, 18.22]], [[-77.35, 8.67], [-74.2, 11.31], [-71.36, 11.54], [-71.35, 10.21], [-68.19, 10.55], [-61.88, 10.72], [-59.1, 8.0], [-55.03, 6.03], [-50.51, 1.9], [-46.57, -0.94], [-37.22, -4.82], [-37.05, -11.04], [-39.58, -18.26], [-45.35, -23.8], [-49.59, -29.22], [-55.67, -34.75], [-56.74, -36.41], [-62.15, -40.68], [-63.46, -42.56], [-66.6, -47.03], [-68.15, -52.35], [-73.7, -52.84]], [[-77.88, 7.22], [-77.93, 2.7], [-80.02, 0.36], [-79.77, -2.66], [-79.45, -7.93], [-73.44, -16.36], [-70.91, -27.64], [-73.59, -37.16], [-72.72, -42.38], [-75.48, -50.38]], [[-74.66, -52.84], [-68.25, -53.1], [-68.15, -55.61]], [[53.51, 73.75], [68.85, 76.54], [57.54, 70.72], [54.43, 73.63]], [[15.14, 79.67], [17.12, 76.81], [13.72, 79.66]], [[-77.88, 7.22], [-79.76, 8.58], [-81.06, 7.82], [-82.85, 8.07], [-84.65, 9.62], [-85.79, 10.44], [-87.17, 12.46], [-87.9, 13.15], [-91.69, 14.13], [-97.26, 15.92], [-103.5, 18.29], [-105.27, 21.42], [-109.44, 25.82], [-112.27, 29.27], [-114.77, 30.91], [-112.76, 27.78], [-110.17, 24.27], [-111.67, 24.48], [-114.47, 27.14], [-115.89, 30.18], [-119.08, 34.08], [-122.95, 38.11], [-124.02, 44.62], [-122.34, 47.36], [-127.85, 52.33], [-133.54, 57.18], [-143.96, 60.0], [-151.72, 59.16], [-153.29, 58.86], [-160.29, 55.64], [-161.8, 55.89], [-158.19, 58.62], [-162.05, 59.27], [-165.73, 62.07], [-160.96, 64.22], [-166.43, 64.69], [-162.49, 66.74], [-162.93, 69.86], [-153.9, 70.89], [-143.59, 70.15], [-132.93, 69.51], [-124.42, 70.16], [-115.25, 68.91], [-108.81, 68.31], [-99.9, 67.81], [-94.23, 69.07], [-92.41, 69.7], [-85.58, 68.78], [-81.39, 67.11], [-89.91, 64.03], [-93.22, 58.78], [-85.01, 55.3], [-78.6, 52.56], [-78.52, 58.8], [-72.91, 62.11], [-66.2, 58.77], [-59.57, 55.2], [-57.13, 51.42], [-68.51, 49.07], [-65.12, 48.07], [-61.04, 45.27], [-67.14, 45.14], [-70.49, 41.8], [-72.29, 41.27], [-74.26, 40.47], [-75.07, 38.78], [-76.54, 38.72], [-76.36, 34.81], [-81.34, 31.44], [-80.13, 25.82], [-82.86, 27.89], [-86.4, 30.4], [-89.41, 29.16], [-94.69, 29.48], [-97.53, 24.99], [-96.29, 19.32], [-90.77, 19.28], [-86.81, 21.33], [-88.3, 18.5], [-88.36, 16.53], [-87.9, 15.86], [-85.68, 15.95], [-83.41, 15.27], [-83.5, 12.87], [-83.59, 10.79]], [[-71.71, 19.71], [-69.25, 19.02], [-70.52, 18.18], [-73.45, 18.22], [-72.78, 19.48]], [[-16.26, 19.1], [-16.26, 22.68], [-13.77, 26.62], [-9.43, 32.04], [-4.59, 35.33], [3.16, 36.78], [10.21, 37.23], [10.15, 34.33], [15.25, 32.27], [20.13, 32.24], [25.16, 31.57], [31.69, 31.43], [34.75, 32.07], [36.15, 35.82], [30.62, 36.68], [26.17, 39.46], [36.91, 41.34], [40.32, 43.13], [39.15, 47.04], [35.51, 45.41], [32.63, 45.52], [29.63, 45.04], [28.99, 41.3], [24.93, 40.95], [23.35, 39.19], [23.15, 36.42], [19.98, 39.69], [18.88, 42.28], [14.9, 45.08], [12.33, 45.38], [16.17, 41.74], [16.87, 40.44], [15.89, 38.75], [12.89, 41.25], [7.85, 43.77], [0.81, 41.01], [-2.15, 36.67], [-7.45, 37.1], [-9.45, 39.39], [-9.39, 43.03], [-1.19, 46.01], [-0.99, 49.35], [6.91, 53.48], [8.09, 56.54], [10.37, 56.61], [10.94, 54.01], [18.62, 54.68], [22.52, 57.75], [24.6, 59.47], [22.87, 59.85], [25.4, 65.11], [17.12, 61.34], [14.1, 55.41], [5.67, 58.59], [16.44, 68.56], [30.01, 70.19], [40.02, 66.27], [37.01, 63.85], [43.02, 66.42], [45.56, 67.57], [54.75, 68.09], [63.5, 69.55], [66.72, 70.71], [72.47, 71.09], [73.92, 66.79], [73.1, 71.45], [77.58, 72.27], [86.01, 74.46], [98.92, 76.45], [107.24, 76.48], [109.4, 74.18], [119.02, 73.12], [129.72, 71.19], [139.15, 72.42], [159.71, 69.72], [170.01, 69.65]], [[180.0, 64.98], [177.36, 62.52], [165.84, 60.16], [162.13, 56.12], [156.42, 51.7], [161.87, 60.34], [154.22, 59.76], [142.2, 59.04], [141.35, 53.09], [135.52, 43.99], [129.97, 41.94], [127.5, 39.32], [128.19, 34.89], [125.69, 37.94], [125.13, 38.85], [121.59, 39.36], [117.53, 38.74], [122.52, 36.93], [121.91, 31.69], [120.4, 27.05], [113.24, 22.05], [108.52, 21.72], [108.88, 15.28], [105.08, 9.92], [100.1, 13.41], [101.02, 6.86], [103.43, 3.38], [101.27, 3.27], [98.99, 7.91], [98.43, 12.03], [94.81, 15.8], [92.08, 21.19], [89.7, 21.86], [85.06, 19.48], [80.03, 15.14], [78.28, 8.93], [74.62, 13.99], [70.47, 20.88], [64.53, 25.24], [55.72, 26.96], [48.94, 30.32], [49.47, 27.11], [50.74, 25.48], [51.79, 24.02], [56.49, 26.31], [59.18, 22.99], [58.03, 20.48], [56.28, 17.88], [52.39, 16.38], [47.35, 13.59], [44.18, 12.59], [42.7, 15.72], [40.94, 19.49], [37.48, 24.29], [34.63, 28.06], [33.92, 27.65], [34.47, 25.6], [36.97, 20.84], [41.18, 14.49], [43.15, 11.46], [48.02, 11.19], [51.13, 11.75], [47.74, 4.22], [40.88, -2.08], [38.8, -6.48], [40.32, -10.32], [39.45, -16.72], [35.18, -21.25], [35.04, -24.48], [32.46, -28.3], [28.22, -32.77], [22.99, -33.92], [18.42, -34.0], [17.06, -29.88], [14.39, -22.66], [11.64, -16.67], [13.74, -11.3], [12.73, -6.93], [8.8, -1.11], [9.4, 3.73], [5.9, 4.26], [-0.51, 5.34], [-6.53, 4.71], [-11.71, 6.86], [-14.58, 10.21], [-16.61, 12.17], [-16.46, 16.14]], [[-180.0, -84.71], [-172.89, -84.06], [-155.19, -85.1], [-150.9, -83.9], [-156.84, -81.1], [-148.06, -79.65], [-158.37, -76.89], [-148.75, -76.91], [-142.79, -75.34], [-133.75, -74.44], [-122.56, -74.5], [-113.3, -74.03], [-104.88, -74.95], [-103.11, -73.73], [-97.69, -73.56], [-88.42, -73.01], [-80.3, -73.13], [-71.62, -73.26], [-67.92, -70.85], [-67.62, -67.72], [-63.63, -64.9], [-57.81, -63.27], [-62.02, -64.8], [-64.88, -67.15], [-62.57, -69.99], [-60.83, -73.7], [-68.45, -76.01], [-75.4, -77.28], [-76.85, -79.51], [-63.26, -81.75], [-51.54, -82.0], [-36.27, -81.12], [-31.62, -79.3], [-31.0, -77.36], [-21.22, -75.91], [-16.11, -73.46], [-9.1, -71.32], [-3.05, -71.29], [5.16, -70.62], [11.95, -70.64], [19.26, -69.89], [27.09, -70.46], [33.87, -68.5], [40.02, -69.11], [47.44, -67.72], [53.61, -65.9], [59.94, -67.41], [66.91, -67.86], [67.95, -70.7], [71.57, -71.7], [76.63, -69.62], [82.05, -67.37], [88.36, -66.48], [95.02, -67.17], [101.58, -66.31], [109.16, -66.84], [115.6, -66.7], [123.22, -66.48], [130.78, -66.43], [135.87, -66.03], [143.06, -66.8], [150.13, -68.56], [158.03, -69.48], [166.11, -70.76], [170.11, -72.89], [164.23, -75.46], [166.6, -78.32], [159.79, -80.95], [169.4, -83.83]], [[-180.0, 68.96], [-170.89, 65.54], [-177.22, 65.52]], [[-155.4, 20.08], [-155.94, 19.06], [-155.4, 20.08]], [[46.68, 44.61], [53.22, 46.23], [51.34, 43.13], [54.01, 41.55], [53.74, 37.91], [48.86, 38.82], [48.58, 41.81]], [[-106.52, 73.08], [-102.09, 69.12], [-113.85, 69.01], [-114.35, 70.6], [-117.87, 72.71], [-108.19, 71.65]], [[-96.02, 80.6], [-85.81, 79.34], [-94.97, 79.37]], [[-91.59, 81.89], [-81.1, 83.02], [-63.68, 82.9], [-69.47, 80.62], [-76.34, 78.18], [-83.17, 76.45], [-84.98, 77.54], [-84.2, 80.21], [-91.59, 81.89]], [[-46.76, 82.63], [-26.52, 82.3], [-20.62, 81.52], [-18.9, 79.4], [-20.67, 75.16], [-22.3, 72.18], [-25.54, 71.43], [-31.78, 68.12], [-40.68, 64.14], [-48.26, 60.86], [-53.97, 67.19], [-54.68, 69.61], [-55.83, 71.65], [-63.39, 76.18], [-73.3, 78.04], [-62.23, 81.32], [-46.6, 81.99]]]

def addCoast(ax,shapes=shapes,col='#3e2c16',alp=0.5):
       for el in shapes:
            if len(el)>=4:
               x,y=[pt[0] for pt in el], [pt[1] for pt in el]
               ax.plot(x,y,color=col,alpha=alp,transform=ccrs.Geodetic(),linewidth=0.5)
########################################################################################

Now we test our function, plotting, for example, all the voyages by Spanish, British and Dutch vessels.

In [ ]:
#here we initalise the geolocator
geolocator = Nominatim()

#we set up the plots
fig = plt.figure(figsize=(8, 12))
gs1 = gridspec.GridSpec(4, 1)
ax1, ax2, ax3, ax4 = fig.add_subplot(gs1[0],projection=ccrs.Robinson()),\
                fig.add_subplot(gs1[1],projection=ccrs.Robinson()),\
                fig.add_subplot(gs1[2],projection=ccrs.Robinson()),\
                fig.add_subplot(gs1[3],projection=ccrs.Robinson())

#this function helps initialise the different plots with Cartopy according to our preferences
def ax_init(ax):
    ax.set_global()
    #the following commands require Cartopy to contact the server and download some information
    #like the coastlines shape, but unfortunately it doesn't seem to be working at the moment, 
    #so I deactivated them, we'll need a bit of fantasy to see the shape of the landmasses 
    #but that's ok, if you download the code locally it should work fine
    #ax.stock_img()
    ##ax.add_feature(cartopy.feature.LAND, facecolor='#dbc79c', alpha=0.5)
    #ax.add_feature(cartopy.feature.OCEAN, facecolor='#dbc79c', alpha=0.5)
    #ax.coastlines(color='#3e2c16', linewidth=0.5, alpha=0.5)
    #we will use here the quick&dirty alternative
    addCoast(ax)  
    ax.outline_patch.set_edgecolor('#51412D')
    ax.outline_patch.set_linewidth(0.5)
    ax.outline_patch.set_alpha(1)   

    
ax_init(ax1), ax_init(ax2), ax_init(ax3), ax_init(ax4)

ax1.set_title("Spanish Travels")
#ships = df1[df1['Nationality']!='Spanish']['ShipName'].unique()
#for ship in ships[:]:
#	plotPath(ship,df1,ax1,'#444444',0.1)

ships = df1[df1['Nationality']=='Spanish']['ShipName'].unique()
for ship in ships[:]:
    plotPath(ship,df1,ax1,'#BB3333',0.3)
    #plotPorts(ship,df2,ax1,'#BB3333','#BB3333')
    
ax2.set_title("British Travels")    
#ships = df1[df1['Nationality']!='British']['ShipName'].unique()
#for ship in ships[:]:
#	plotPath(ship,df1,ax2,'#444444',0.1)

ships = df1[df1['Nationality']=='British']['ShipName'].unique()
for ship in ships[:]:
    plotPath(ship,df1,ax2,'#3333BB',0.3)
    #plotPorts(ship,df2,ax2,'#3333BB','#3333BB')
    
ax3.set_title("Dutch Travels")    
#ships = df1[df1['Nationality']!='Dutch']['ShipName'].unique()
#for ship in ships[:]:
#	plotPath(ship,df1,ax3,'#444444',0.1)

ships = df1[df1['Nationality']=='Dutch']['ShipName'].unique()
for ship in ships[:]:
    plotPath(ship,df1,ax3,'#BBBB33',0.3)
    #plotPorts(ship,df2,ax3,'#3333BB','#3333BB')

ax4.set_title("French Travels")    
#ships = df1[df1['Nationality']!='Dutch']['ShipName'].unique()
#for ship in ships[:]:
#	plotPath(ship,df1,ax3,'#444444',0.1)

ships = df1[df1['Nationality']=='French']['ShipName'].unique()
for ship in ships[:]:
    plotPath(ship,df1,ax4,'#44AA44',0.3)
    #plotPorts(ship,df2,ax3,'#33BB33','#33BB33')

From these plots, one can then try to make sense of the huge amount of data contained in the logbooks dataset.

While the Spanish empire mostly focused on central and south America, the British went further north and repeatedly visited India and south-east Asia. Similarly, the Dutch navy interacted with most of south America, central Africa and south-east Asia, while the French limited themselves to central and north America. These assumptions are relative to the number of entries in the logbook (higher density of lines suggests that most of the entries come from Dutch travels) and represent an incomplete description of the intercontinental voyages of these maritime powers, but they are in line with what we all learned from history books. One could then further analyse the evolution of these naval routes over time, the growth of commercial ports, battles between rival nations and more. This logbook is indeed a historical testimony of one of the most prosperous periods of maritime exploration and commerce for European nations. 

Putting aside all this interesting historical data, we will now focus on the weather logs and see what we can learn.

In [ ]:
#we import the library
from wordcloud import WordCloud
from functools import reduce

fig = plt.figure(figsize=(6, 12))
gs1 = gridspec.GridSpec(3, 1)
ax1, ax2, ax3 = fig.add_subplot(gs1[0]),\
                fig.add_subplot(gs1[1]),\
                fig.add_subplot(gs1[2])
        
df3=df.dropna(subset = ['WindForce','StateSea','Weather'])
df3=df3[df3['Nationality']=='British']

def plotWordCloud(column, df, ax):
    repls = ("weather" , ""), ("sea" , ""), ("water", ""), ("air", ""), ("  ", " ")
    text=" ".join(df[column].tolist())
    text=reduce(lambda a, kv: a.replace(*kv), repls, text)
    wordcloud = WordCloud(background_color="white").generate(text)
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.axis("off")

plotWordCloud('Weather', df3, ax1)
plotWordCloud('StateSea', df3, ax2)
plotWordCloud('WindForce', df3, ax3)

## Weather Logbooks ##

Information on the climate conditions met by the sailors can be found in different columns of the .csv file. We can get information on the sea (StateSea), the winds (WindForce) and general weather (Weather) We can first try to extract the information we need directly from there. A qualitative analysis of the frequency of appearance of specific words will tell us if this is a good idea and something more. Here we will use word cloud, a library that does exactly that with just a few lines of code, but we will use only English words for simplicity (although armed with a translator algorithm or a dictionary one could repeat the analysis for other languages).

Although visually interesting, this doesn't tell us much about the climate during the voyages, but it shows that the task ahead is harder than it seemed at the beginning. For this analysis, we don't really care if a ship met a light breeze or a fresh one, and we need a way to join different synonyms and apply a more systematic classification. The task becomes even more daunting when considering that most of the logbook is in different languages!

Luckily for us, somebody did most of the work already and categorised part this information in a few columns, where the presence or absence of weather phenomena like rain, snow or storms is registered for each logbook entry.

## KDE ##

Mapping directly all the entries on the map could result in a very confusing plot, thus we will use instead a Kernel Density Estimation algorithm to infer the probability density of such phenomena in different geographic and temporal points and extract a density map. This method simply estimates the probability density function of a population starting from a limited subset of data: the histogram of such points as a function of the features of interest is smoothed by kernel functions. Different parameters have to be chosen and can affect the quality of the result, such as the bandwidth (which tunes the level of smoothing) and the shape of the kernel functions, which need to be positive, with zero mean and to integrate to one.

In [ ]:
#we will use kde as implemented in sklearn
import types
from matplotlib import animation
from sklearn.neighbors import KernelDensity
from sklearn.grid_search import GridSearchCV
df4=df.dropna(subset = ['Lon3', 'Lat3', 'Month', 'Year'])

Again, we first build a function to calculate and plot the KDE for a given set of geographical points.

In [ ]:
#we define a KDE function, following the implementation found at 
#https://jakevdp.github.io/blog/2013/12/01/kernel-density-estimation/

def plotKDE(x, y, ax, xbins=100j, ybins=100j, cmap=plt.get_cmap('Blues'), **kwargs): 
	# first we create grid of sample locations (default: 100x100)
	xx, yy = np.mgrid[np.min(x):np.max(x):xbins, np.min(y):np.max(y):ybins]
	#we reshape the data 		
	xy_sample = np.vstack([yy.ravel(), xx.ravel()]).T
	xy_train  = np.vstack([y, x]).T
	#now if an adequate bandwidth is known it can be used directly, otherwise
	#gridsearch can find the best value for each dataset     
	kde_skl = KernelDensity(bandwidth=1, **kwargs)
	#20-fold cross-validation
	#kde_skl = GridSearchCV(KernelDensity(), {'bandwidth': np.linspace(0.1, 1.0, 30)}, cv=20) 
	kde_skl.fit(xy_train)
	# score_samples() returns the log-likelihood of the samples
	#print('Best Bandwidth: '+str(kde_skl.best_params_['bandwidth']))
	#kde_skl = kde_skl.best_estimator_
	z = np.exp(kde_skl.score_samples(xy_sample))
	data=np.reshape(z, xx.shape)
	clevs=np.arange(np.max(data)/10,np.max(data),np.max(data)/10)
	cf=ax.contourf(xx, yy, data, clevs, alpha=0.5, cmap=cmap,\
                   extend='max', transform=ccrs.PlateCarree())

And then we apply it to visualise the climate information found in the logbook. Let's first apply it to all the data of a given phenomenon independently on the time it was taken (and ignore variations dependent on season and year).

From the output grid obtained from KDE, we can plot nice density contour maps where a higher intensity of the colour represents a higher value of relative density.

In [ ]:
#df4=df[['Lon3', 'Lat3', 'Gusts','Rain','Fog','Snow','Thunder','Hail','SeaIce']]
fig = plt.figure(figsize=(10, 12))
gs1 = gridspec.GridSpec(4, 2)
ax1, ax2, ax3, ax4, ax5, ax6, ax7 = fig.add_subplot(gs1[0],projection=ccrs.Robinson()),\
                fig.add_subplot(gs1[1],projection=ccrs.Robinson()),\
                fig.add_subplot(gs1[2],projection=ccrs.Robinson()),\
                fig.add_subplot(gs1[3],projection=ccrs.Robinson()),\
                fig.add_subplot(gs1[4],projection=ccrs.Robinson()),\
                fig.add_subplot(gs1[5],projection=ccrs.Robinson()),\
                fig.add_subplot(gs1[6],projection=ccrs.Robinson())            
                
ax_init(ax1), ax_init(ax2), ax_init(ax3), ax_init(ax4),\
ax_init(ax5), ax_init(ax6), ax_init(ax7)

ax1.set_title('Total Gusts KDE')
plotKDE(df4[df4.Gusts==1]['Lon3'].tolist(),df4[df4.Gusts==1]['Lat3'].tolist(),\
        ax1,cmap=plt.get_cmap('Blues'))

ax2.set_title('Total Rain KDE')
plotKDE(df4[df4.Rain==1]['Lon3'].tolist(),df4[df4.Rain==1]['Lat3'].tolist(),\
        ax2,cmap=plt.get_cmap('Reds'))

ax3.set_title('Total Fog KDE')
plotKDE(df4[df4.Fog==1]['Lon3'].tolist(),df4[df4.Fog==1]['Lat3'].tolist(),\
        ax3,cmap=plt.get_cmap('Greys'))

ax4.set_title('Total Snow KDE')
plotKDE(df4[df4.Snow==1]['Lon3'].tolist(),df4[df4.Snow==1]['Lat3'].tolist(),\
        ax4,cmap=plt.get_cmap('YlGnBu'))

ax5.set_title('Total Thunder KDE')
plotKDE(df4[df4.Thunder==1]['Lon3'].tolist(),df4[df4.Thunder==1]['Lat3'].tolist(),\
        ax5,cmap=plt.get_cmap('Oranges'))

ax6.set_title('Total Hail KDE')
plotKDE(df4[df4.Hail==1]['Lon3'].tolist(),df4[df4.Hail==1]['Lat3'].tolist(),\
        ax6,cmap=plt.get_cmap('Purples'))

ax7.set_title('Total SeaIce KDE')
plotKDE(df4[df4.SeaIce==1]['Lon3'].tolist(),df4[df4.SeaIce==1]['Lat3'].tolist(),\
        ax7,cmap=plt.get_cmap('BuGn'))

The maps are very interesting, they give us a rough idea of the climate conditions met by these travellers in those data: among the windiest and rainiest regions are the westernmost point of Africa, where most of the storms are also concentrated. This tell's us that records were probably taken during rain season. Similar is Britain, where most hail and fog was also found (what a surprise!). Snow and was observed South of Tierra del Fuego and along the Canadian coasts, where some ships were met with a frozen sea. 

The observations are proportional to the number of travel and are summed over all months and years found in the logbook, so the information we can extract is limited. Let's see if there are seasonal variations for example in the amount of rain.

In [ ]:
#We now create functions to group the logs according to groups of 10 years
#(starting from the first year found in the logbook) and two main seasons (Winter-Summer) 

maxyear=df4['Year'].max()
minyear=df4['Year'].min()
def year_xform(df):
	year=minyear
	while year<maxyear:
	    if df >= year and df<year+25 : return year
	    year=year+25	
        
def season_xform(df):
    if df>3 and df<=10: 
            return 1
    else:
            return 0
        
df4['Season'] = df4['Month'].map(season_xform)
df4['25years'] = df4['Year'].map(year_xform)

#We start by plotting the seasonal difference
fig = plt.figure(figsize=(10, 12))
gs1 = gridspec.GridSpec(1, 2)
ax1, ax2 = fig.add_subplot(gs1[0],projection=ccrs.Robinson()),\
                fig.add_subplot(gs1[1],projection=ccrs.Robinson())            
                
ax_init(ax1), ax_init(ax2)

ax1.set_title('Winter Rain KDE')
plotKDE(df4[(df4.Season==0) & (df4.Rain==1)]['Lon3'].tolist(),df4[(df4.Season==0) & (df4.Rain==1)]['Lat3'].tolist(),\
        ax1,cmap=plt.get_cmap('Blues'))

ax2.set_title('Summer Rain KDE')
plotKDE(df4[(df4.Season==1) & (df4.Rain==1)]['Lon3'].tolist(),df4[(df4.Season==1) & (df4.Rain==1)]['Lat3'].tolist(),\
        ax2,cmap=plt.get_cmap('Reds'))


grouped=df4.groupby('25years')
num=len(grouped)

#Now, since we don't know how many groups we have, we can automatise the 
#building of the plot, so that it works for any number of plots

fig = plt.figure(figsize=(10, 12))
gs1 = gridspec.GridSpec(int(num/2)+1, 2)
ax=[]

i=0
for name,group in grouped:
    ax.append(fig.add_subplot(gs1[i],projection=ccrs.Robinson()))
    ax_init(ax[i])
    ax[i].set_title('Rain KDE '+str(name)+'-'+str(name+25))
    plotKDE(group[group.Rain==1]['Lon3'].tolist(),group[group.Rain==1]['Lat3'].tolist(),\
        ax[i],cmap=plt.get_cmap('Reds'))
    i=i+1
    
##########################################################################
#The following part is in case you want to plot how the density 
#changes every X years as an animation

#we define a function that only calculates the kde and does not plot it
#since it will be plotted with the animation
#def kde2D(x, y, xbins=100j, ybins=100j, **kwargs): 
#	# first we create grid of sample locations (default: 100x100)
#	xx, yy = np.mgrid[np.min(x):np.max(x):xbins, np.min(y):np.max(y):ybins]
#	#we reshape the data 		
#	xy_sample = np.vstack([yy.ravel(), xx.ravel()]).T
#	xy_train  = np.vstack([y, x]).T
#	#now if an adequate bandwidth is known it can be used directly, otherwise
#	#gridsearch can find the best value for each dataset     
#	kde_skl = KernelDensity(bandwidth=1, **kwargs)
#	#20-fold cross-validation
#	#kde_skl = GridSearchCV(KernelDensity(), {'bandwidth': np.linspace(0.1, 1.0, 30)}, cv=20) 
#	kde_skl.fit(xy_train)
#	# score_samples() returns the log-likelihood of the samples
#	#print('Best Bandwidth: '+str(kde_skl.best_params_['bandwidth']))
#	#kde_skl = kde_skl.best_estimator_
#	z = np.exp(kde_skl.score_samples(xy_sample))
#	return xx, yy, np.reshape(z, xx.shape)
#
#maxyear=df4['Year'].max()
#minyear=df4['Year'].min()
#def year_xform(df):
#	year=minyear
#	while year<maxyear:
#	    if df >= year and df<year+10 : return year
#	    year=year+10
#df4['Decade'] = df4['Year'].map(year_xform)
#grouped=df4.groupby('Decade')
#gustsX,gustsY=[],[]
#groupname=[]
#for name,group in grouped:
#	groupname.append(name)
#	gustsX.append(group[group['Rain'] ==1]['Lon3'].tolist())
#	gustsY.append(group[group['Rain'] ==1]['Lat3'].tolist())
##	print len(group[group['Gusts'] ==1]['Lon3'].tolist()), name

#cmap = plt.get_cmap('Blues')

#listXX,listYY,listData=[],[],[]
#for i in range(len(gustsX)):
#	xx,yy,data=kde2D(gustsX[i], gustsY[i])
#	listXX.append(xx)
#	listYY.append(yy)
#	listData.append(data)
   
#def init():
#    cf=ax.contourf([],[],[])	
##    cf.set_data([], [], [])
#    return cf

#def animate(i):
#	xx=listXX[i]
#	yy=listYY[i]
#	data=listData[i]
#	clevs=np.arange(np.max(data)/10,np.max(data),np.max(data)/10)
#    #Unfortunately countorf and animation don't really work together and 
#    #it is not possible to remove the previous plot for every new
#    #frame of the animation, so we'll have to replot everyhing
#    #including the background map for each frame (extremely slow)
#    #for coll in cf.collections:
#    # 	plt.gca().collections.remove(coll) 
#    #for obj in ax.findobj(match=None, include_self=True):
#    #   ax.findobj(match = type(ax.contourf(xx,yy,data))):
#    #  		obj.remove()
#	plt.cla()
#	ax = plt.axes(projection=ccrs.Robinson())
#	ax.set_global()
#	ax.outline_patch.set_edgecolor('#3e2c16')
#	ax.outline_patch.set_linewidth(0.5)
#	ax.outline_patch.set_alpha(0.5)
#	#ax.stock_img()
#	#ax.add_feature(cartopy.feature.OCEAN, facecolor='#dbc79c', alpha=0.5)
#	#ax.add_feature(cartopy.feature.LAND, facecolor='#dbc79c', alpha=0.5)
#	#ax.coastlines(color='#3e2c16', linewidth=0.5, alpha=0.5)

#	cf=ax.contourf(xx, yy, data, clevs, alpha=0.5, cmap=cmap, extend='max', transform=ccrs.PlateCarree())	
#	plt.title('%d' % groupname[i])
#	return cf    

#output = animation.FuncAnimation(plt.gcf(), animate, frames=len(listData), interval=50, repeat=True)
##############################################################

These results are not exactly what expected. Although a small variation is observed between the season (with a slight increase in rainfalls registered close to the African coast) this is most likely to be due to the lower number of entries during the winter (when fewer vessels would probably due to the cold temperature in the northern Hemisphere) and summer. 

Similarly, there are few entries from the first 50 years (top two graphs of the second figure) which makes them unreliable for our analysis. It would appear that an increase of precipitation was observed starting from the second part of the XXVIII century, in particular in central Africa and South-East Asia, while fewer rain events were registered around the British isles at the turn of the century. Again, however, the contour map follows closely the vessels paths, giving us a glimpse of how new routes started to be travelled later in the century, something that we could have however easily obtained by looking more in depth at our initial analysis of the voyages, rather than giving us information about the weather evolution.

## Neural Network ##

Now let's get down to business. We will now implement a very elementary neural network with Theano library to predict the weather conditions according to a few factors, such as geographical position and date. We will feed 80% of the logbook dataset as training the remaining 10% will be used for validation and 10% for testing. This ratio follows a general rule of thumb, but it should be optimised to the specific system.

In [ ]:
#We import the theano libraries we will need and the metric 
#that will be used as cost function
import theano
import theano.tensor as T
import theano.tensor.nnet as nnet
from sklearn.metrics import log_loss

Now we will import the data, clean it and normalise it. We keep only the information we need, geometric position, month and year,  as we can expect to lose little information by dropping day and hours of the log entries. We also add an extra column for the sunny weather,  that will be filled when no other weather phenomenon has been observed or registered on the logbook. This is a good assumption, given the results observed in the word clouds, where the most frequent words described fine weather and calm sea. 

In [ ]:
#we take only the information we need 
df5=df[['Lon3', 'Lat3', 'Year', 'Month', 'Gusts', 'Rain', 'Fog', 'Snow', 'Thunder', 'Hail', 'SeaIce']]
df5=df5.dropna()

#we define a function to check if the weather was sunny
def sunny(row):
    if row['sum'] == 0:
        val = 1
    else:
        val= 0
    return val

#and we apply it to the dataset
df5['sum'] = df5[['Gusts', 'Rain', 'Fog', 'Snow', 'Thunder', 'Hail', 'SeaIce']].sum(axis=1)
df5['Sun'] = df5.apply(sunny, axis=1)

#then we normalise the features, so that they have zero mean and variance equal to one
#this will help the learning algorithm, since each feature will weight the same
for column in ['Lon3', 'Lat3', 'Year', 'Month']:
        df5[column]=(df5[column]-df5[column].mean(axis=0))/df5[column].std(axis=0)

#now we split the dataset in training, testing and validation
#normaally, it is good practice to make sure to take subset that correctly
#reproduce the distribution of the data, but in this case, given
#that the dataset is really big, we will just take random rows
#dfTrain=df5.sample(frac=0.8,random_state=200)
#dfVal=df5.drop(dfTrain.index).sample(frac=0.5,random_state=5)
#dfTest=df5.drop(dfTrain.index).drop(dfVal.index)

#WARNING: here we take only a small sample to avoid the 
#notebook to fail because of the time limit, 
#the result will not be representative of the actual efficacy
#of the neural network
#but you can try with the whole notebook on your pc
dfTrain=df5.sample(n=1000,random_state=20)
dfVal=df5.drop(dfTrain.index).sample(n=125,random_state=50)
dfTest=df5.drop(dfTrain.index).drop(dfVal.index).sample(n=125,random_state=80)

#we are now (almost) ready to go, let's change the format of the data and put it into matrices
xTrain=dfTrain[['Lon3', 'Lat3', 'Year', 'Month']].as_matrix() 
yTrain=dfTrain[['Gusts', 'Rain', 'Fog', 'Snow', 'Thunder', 'Hail', 'SeaIce', 'Sun']].as_matrix()
xVal=dfVal[['Lon3', 'Lat3', 'Year', 'Month']].as_matrix() 
yVal=dfVal[['Gusts', 'Rain', 'Fog', 'Snow', 'Thunder', 'Hail', 'SeaIce', 'Sun']].as_matrix()
xTest=dfTest[['Lon3', 'Lat3', 'Year', 'Month']].as_matrix() 
yTest=dfTest[['Gusts', 'Rain', 'Fog', 'Snow', 'Thunder', 'Hail', 'SeaIce', 'Sun']].as_matrix()

Now we start building the Neural Network with Theano. We will use just one hidden layer, which is likely to be not enough to crunch through all the notebook data, but we are trying to keep things clear and easy for our very basic toy model. 

This model is mostly based on a number of tutorials that can be found at the following links: 
[one][1], [two][2], [three][3] and [four][4]. I suggest you to have a look at them if you want to learn more.

  [1]: http://iamtrask.github.io/2015/07/12/basic-python-network/
  [2]: http://outlace.com/Beginner-Tutorial-Theano/
  [3]: http://deeplearning.net/tutorial/mlp.html
  [4]: http://stackoverflow.com/questions/38708956/training-mlp-in-theano

In [ ]:
np.random.seed(1)

#we first define our input tensors
x = T.dvector()
y = T.dvector()

#and we build a layer function, that takes an input vector x
#and a vector containing the weights w
#we also add a bias node b in case the activation function,
#here a sigmoid, needs to be rigidly shifted
def layer(x, w):
    b = np.array([1], dtype=theano.config.floatX)
    new_x = T.concatenate([x, b])
    m = T.dot(w.T, new_x) 
    h = nnet.sigmoid(m)
    return h

#we define a function to update the weights through a gradient
#descent optimisation
def grad_desc(cost, theta):
    alpha = 0.2 #learning rate which defines how quickly 
    #the weights will update following the gradient
    return theta - (alpha * T.grad(cost, wrt=theta))

#we now define the weights (with the correct shapes) for the input
#and the hidden layer and we initialise them randomly
theta1 = theano.shared(np.array(np.random.rand(xTrain.shape[1]+1,xTrain.shape[1]+1),\
                                dtype=theano.config.floatX)) 
theta2 = theano.shared(np.array(np.random.rand(xTrain.shape[1]+2,yTrain.shape[1]),\
                                dtype=theano.config.floatX))

#we build the hidden and ouput layers
hid1 = layer(x, theta1)
out1 = layer(hid1, theta2) 

#we define cost and accuracy functions
#cost = T.sum((out1-y)**2) #simple cost expression
cost = T.mean(T.nnet.binary_crossentropy(out1, y))
#in this case we will get probabilites as ouputs 
#instead of binary results, for this reason it's
#more sensible to use .isclose() to quantify the accuracy,
#alhtouhgh one could simply define a cut-off and make
#all the outputs binary 
#acc= T.mean(T.eq(out1, y)) 
acc= T.mean(T.isclose(out1, y, rtol=1e-01, atol=1e-02))

#now it's time to define the theano functions, 
#one for each phase of the learning
#first we train our network, adjusting the weights
#according to the gradient descent and the cost function
train = theano.function(inputs=[x, y], outputs=[cost,acc], updates=[
        (theta1, grad_desc(cost, theta1)),
        (theta2, grad_desc(cost, theta2))])
#the validation function is very similar, with the difference 
#the weights won't be updated at each iteration
valid = theano.function(inputs=[x, y], outputs=[cost,acc])
#finally in the test function we are only interested
#in evaluating the accuracy
test = theano.function(inputs=[x, y], outputs=acc)
#we also define a run_forward function, which would allow
#us to apply the neural network to data to make actual predictions
run_forward = theano.function(inputs=[x], outputs=out1)

Good, the network is now set up, we just need to run it. Theano will automatically compile the functions and the layers we defined once we call them. The cycles (epochs) through which the learning proceeds usually are divided in a training phase, where the weights are updated following the gradient of the cost function, and in a validation phase in which the network is tested against the validation set.

This set is used to check that the network we built is actually working, by testing its accuracy on a set that was not used in the training: if the accuracy of the test set increases, but that of the validation set does not, it means that the model is most likely overfitting the training set, and should be changed accordingly (for example by adding or removing a number of hidden layers, or by changing cost or activation function).
Usually one should set the cycle to continue until a certain threshold of accuracy is reached for the validation, or until it has converged to a satisfactory value (if that's the best our model can do). In this case, we will just set a fixed number of epochs since we don't know which accuracy to expect.

Finally, the test set does what the name implies, if everything went as expected, the cost and accuracy converged for both training and validation sets, the network is tested on the remaining number of data and the accuracy is evaluated. If its value is satisfactory we have our working neural network.	

In [ ]:
#we choose a fixed number of epochs
epochs=2500 
#we will use these vectors to plot how loss
#and accuracy change during the training 
meanTrain, meanVal=[],[] 

for i in range(epochs+1):
    trainOut,valOut=[],[]
    for k in range(len(xTrain)):
        trainOut.append(train(xTrain[k], yTrain[k])) 
    for k in range(len(xVal)):
	    valOut.append(valid(xVal[k], yVal[k]))
    meanTrain.append(np.mean([[x,y] for x,y in trainOut], axis=0))
    meanVal.append(np.mean([[x,y] for x,y in valOut], axis=0))
    if i % 500 == 0: #we print every 500th cycle 
	    print(' Mean Cost Training: %s' % meanTrain[i][0])
	    print(' Mean Accuracy Training: %s' % meanTrain[i][1])
	    print(' Mean Cost Validation: %s' % meanVal[i][0])
	    print(' Mean Accuracy Validation: %s' % meanVal[i][1])
	    print('---------------------------------------')
    #to improve the result it may be worth shuffling the training
    #data after each iteration, but it doesn't seem 
    #to have much effect here
    #xTrain, yTrain = shuffle(xTrain, yTrain, random_state=0)

In [ ]:
#Now we plot the trends, it will help us see if 
#there's an improvement during the training
fig = plt.figure(figsize=(12, 6))
gs1 = gridspec.GridSpec(1, 2)
ax1, ax2 = fig.add_subplot(gs1[0]),\
                fig.add_subplot(gs1[1])  

ax1.plot([x for x,y in meanTrain], color='#31427e', label='Training')
ax1.plot([x for x,y in meanVal], color='#7e3131', label='Validation')
ax2.plot([y for x,y in meanTrain], color='#31427e', label='Training')
ax2.plot([y for x,y in meanVal], color='#7e3131', label='Validation')

ax1.set_xlabel('Epoch'), ax2.set_xlabel('Epoch')    
ax1.set_ylabel('Cost'), ax2.set_ylabel('Accuracy')
ax1.legend(frameon=False),ax2.legend(frameon=False)

In [ ]:
#And finally we calculate the final accuracy
print('==================================')
finAcc=[]
for k in range(len(xTest)):
	finAcc.append(test(xTest[k], yTest[k]))
print( ' Final Accuracy: %s' % np.mean(finAcc))
print('==================================')

These results are obviously not very exciting. The neural network we built has blindly applied to a very limited subset of data, which is not guaranteed to be representative of the underlying distribution of weather classes (a good sign of this is that the accuracy is strongly dependent on the set of data we take, repeating the calculation with a different random state changes completely the accuracy value). More in general, one could not expect such a simple toy model to work accurately with a complex dataset like the logbooks. At this point, one should take the results obtained working with a much larger set of data (if not the whole notebook) and analyse the behaviour of accuracy and cost on both the training and validation sets, adjust the number of hidden layers, the learning rate, the functions, until a satisfactory output is obtained. Nevertheless, it's a good exercise and hopefully will be helpful to other beginners like me who want an easy approach to Neural Networks. 